In [1]:
'''
Run Boston housing data on Gradient descent with different learning rates. values. Run the training data to get the theta values 
for all learning rates which has minimum cost. For each of the theta values calculate the Correlation coefficient. Theta which 
gives the maximum r2 score will be used to predict test data and cost of test data.


'''

In [4]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import r2_score
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

In [5]:

data=pd.read_csv('https://raw.githubusercontent.com/rromanss23/Machine_Leaning_Engineer_Udacity_NanoDegree/master/projects/boston_housing/housing.csv')

In [6]:
#Standardize the data
data=(data - data.mean())/data.std()

In [7]:
#Insert a column with '1'. This is to keep the values of intercept(c) same across interation. Also to have appropiate dimension
#for matrix calculation
data.insert(0,'ones',1)


In [8]:
#Get the X data
X=data.iloc[:,0:4]

In [9]:
#Extract y data 
y=data.iloc[:,-1]

In [10]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [11]:
#Convert the dataframe and series into ndarray.
xtrain=np.array(x_train.values)
ytrain=np.array(y_train.values)
xtest=np.array(x_test.values)
ytest=np.array(y_test.values)

In [12]:
'''
Generate random values for theta at the begining.The first values will represent intercept (c) value.The remaining are slope(m)
values for the equation of the line y=mx + c

'''
theta = np.random.rand(4)


In [13]:
alpha=[0.01,0.1,1,5,10]  # Represents the learning curve.

In [14]:
'''
Create data frame to hold theta values for a given alpha and at a given threshold value of cost.Dataframe also to hold cost for
every alpha value till a given threshold of cost

'''
theta_data=np.zeros(shape=(len(alpha),len(theta)))
df_theta=pd.DataFrame(theta_data,index=alpha,columns=[0,1,2,3])
df_cost=pd.DataFrame(columns=[alpha])


In [15]:
for val in alpha:
   
    i=0
    cost=0
    cost_list=[]
    theta = np.random.rand(4)
    temp=np.zeros(theta.shape)
    run=True
   

    while run:
    
        pred=np.dot(xtrain,theta) # y=mx+c
       
        error=pred-ytrain  # predicted y - actual y
        cost=np.sum(np.power(error,2))/len(xtrain)                 #Calculate the cost/MSE at each epoch.
        cost_list.append(cost)                                  #Append the cost/MSE in a list.
        
        for j in range(0,len(theta)):                               #Iterating through len(theta) = Number of columns in X data
       
            if j==0:                                                #Cal derivative of c only for 1st value of theta
                intercept=np.sum(np.dot(error,2))/len(xtrain)       #Cal deraitive of intercept(c)=1/len(x) *Sum((2(mx +c - actual y))
                temp[j]=theta[j] - (val*intercept)                #Calculating GD for intercept(c).
                    
            else:
                
                term=np.multiply(error ,xtrain[:,j])                           #Cal derivative of slope(m) = 1/len(x) *Sum((2*X(mx +c -actual y))
                temp[j]=theta[j] - 2*((val / len(xtrain)) * np.sum(term))    #Calculate GD for slope(m).
               
        theta=temp
       
            
        if (i!=0) and (cost_list[i-1] - cost_list[i] <0.0001):
           
            df_theta.loc[val]=theta
            df_cost[val]=pd.Series(cost_list)
            run=False
        i=i+1
   

In [16]:
'''
Calculate correlation coefficient - Pearson Correlation

'''
sum_r_lst=[]
df_coeff=pd.DataFrame(index=alpha,columns=['R-coeff'])
df_new=pd.DataFrame(index=alpha)
for i in range((df_theta.shape[0])):
    coeff=0
    
    #Using iloc to access the values of the current row denoted by "i"
                                                   
    row_val= df_theta.iloc[i, :]
    row_val=np.array(row_val.values)                                #Converting series to numpy
    new_pred=np.dot(xtrain,row_val)                                 #Predicting xtrain data with different theta values obtained above
    coeff=r2_score(ytrain,new_pred,multioutput='variance_weighted') #use r2_score to the correlation coefficient
    df_coeff.at[df_theta.index[i]]=coeff                            #Populate the coefficient against each theta value
    
    sum_r=0
   


In [18]:
'''
Alpha with the corresponding theta values that gives the highest correlation coefficent will be used to predict the test data.

'''

max_coeff=max(df_coeff['R-coeff'])
ida=df_coeff[['R-coeff']].astype(float).idxmax()[0] #Get the row index number for max coefficient value
row_val1= df_theta.loc[ida]                         #Get the corresponding theta value for highest coefficient value
theta_best=np.array(row_val1.values)                #Convert series of theta to numpy array
pred_test=np.dot(xtest,theta_best)                  #Predict test data with best theta obtained above
err_test=pred_test-ytest                            # Get the error for test data - predicted y - actual y
cost_test=np.sum(np.power(err_test,2))/len(xtest)                 #Calculate the cost/MSE for test data.
print('Cost for test data' ,cost_test)


Cost for test data 0.2610138470645927
